# Long Alternative Attention Dataset Preparation - Sum+Max Embeddings

**Purpose:** Prepare dataset using BOTH sum and max of concept similarities.

**Key Features:**
- **Sum-based**: Captures posts relevant to MULTIPLE concepts
- **Max-based**: Captures posts expert in SINGLE concepts
- **Concatenation**: 384 + 384 = **768-dimensional embeddings**
- Posts retrieved **separately** for each method

**Runtime:** ~80-100 minutes (2x the original due to dual retrieval)

This notebook:
1. Loads training and test data from XML files
2. Uses SBERT to retrieve top-50 posts per subject with **SUM-based** scoring
3. Uses SBERT to retrieve top-50 posts per subject with **MAX-based** scoring
4. Pools both sets using their respective attention mechanisms
5. **Concatenates** sum and max embeddings: **768 dims total**
6. Saves everything to `data/processed/long_alternative_attention_pipeline/`

## Section 0: Configuration & Setup

In [1]:
# Imports
import os
import glob
import re
import zipfile
import tempfile
import shutil
import json
import time

import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET

import torch
from sentence_transformers import SentenceTransformer, util
from sklearn.model_selection import train_test_split
from tqdm import tqdm

print("✓ All imports successful")

/opt/miniconda3/envs/38concept_embedding/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


✓ All imports successful


In [2]:
# Set random seeds for reproducibility
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

print(f"✓ Random seed set to {SEED}")

✓ Random seed set to 42


In [3]:
# Detect device (MPS/CUDA/CPU)
if torch.backends.mps.is_available():
    DEVICE = "mps"
    print("✓ Using MacBook GPU (MPS)")
elif torch.cuda.is_available():
    DEVICE = "cuda"
    print("✓ Using CUDA GPU")
else:
    DEVICE = "cpu"
    print("⚠ Using CPU (will be slow)")

✓ Using MacBook GPU (MPS)


In [4]:
# Define paths
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
DATA_RAW = os.path.join(PROJECT_ROOT, "data/raw")
DATA_PROCESSED = os.path.join(PROJECT_ROOT, "data/processed")

# Training data paths
POS_DIR = os.path.join(DATA_RAW, "train/positive_examples_anonymous_chunks")
NEG_DIR = os.path.join(DATA_RAW, "train/negative_examples_anonymous_chunks")

# Test data paths
TEST_DIR = os.path.join(DATA_RAW, "test")
TEST_LABELS = os.path.join(TEST_DIR, "test_golden_truth.txt")

# Concept labels
CONCEPTS_FILE = os.path.join(DATA_PROCESSED, "merged_questionnaires.csv")

# Output directory - LONG ALTERNATIVE PIPELINE (768-dim embeddings)
SAVE_DIR = os.path.join(DATA_PROCESSED, "long_alternative_attention_pipeline")
os.makedirs(SAVE_DIR, exist_ok=True)

print("✓ Paths configured")
print(f"  Project root: {PROJECT_ROOT}")
print(f"  Data save dir: {SAVE_DIR}")
print(f"\n  NOTE: This creates 768-dim embeddings (sum+max concatenation)")

✓ Paths configured
  Project root: /Users/gualtieromarencoturi/Desktop/thesis/Master-Thesis-CEM-Depression-etc-case-study
  Data save dir: /Users/gualtieromarencoturi/Desktop/thesis/Master-Thesis-CEM-Depression-etc-case-study/data/processed/long_alternative_attention_pipeline

  NOTE: This creates 768-dim embeddings (sum+max concatenation)


In [5]:
# Define 21 BDI-II concept names
CONCEPT_NAMES = [
    "Sadness", "Pessimism", "Past failure", "Loss of pleasure",
    "Guilty feelings", "Punishment feelings", "Self-dislike", "Self-criticalness",
    "Suicidal thoughts or wishes", "Crying", "Agitation", "Loss of interest",
    "Indecisiveness", "Worthlessness", "Loss of energy", "Changes in sleeping pattern",
    "Irritability", "Changes in appetite", "Concentration difficulty",
    "Tiredness or fatigue", "Loss of interest in sex"
]
N_CONCEPTS = len(CONCEPT_NAMES)

print(f"✓ Defined {N_CONCEPTS} BDI-II concepts")

✓ Defined 21 BDI-II concepts


In [6]:
# Hyperparameters
HYPERPARAMS = {
    "k_posts": 50,              # Top-k posts per subject per method
    "sbert_model": "all-MiniLM-L6-v2",
    "embedding_dim": 384,       # Dimension per method (sum or max)
    "final_embedding_dim": 768, # 384 (sum) + 384 (max) = 768
}
# =========================
# DEBUG / SANITY CHECK CONFIG
# =========================
DEBUG = True
DEBUG_N_SUBJECTS = 3          # how many subjects to inspect
DEBUG_TOP_N_POSTS = 5         # how many top posts to print
DEBUG_PRINT_CONCEPTS = True   # print per-concept similarity stats

print("✓ Hyperparameters configured:")
for k, v in HYPERPARAMS.items():
    print(f"  {k}: {v}")
print(f"\n  Final embedding: {HYPERPARAMS['final_embedding_dim']} dims = {HYPERPARAMS['embedding_dim']} (sum) + {HYPERPARAMS['embedding_dim']} (max)")

✓ Hyperparameters configured:
  k_posts: 50
  sbert_model: all-MiniLM-L6-v2
  embedding_dim: 384
  final_embedding_dim: 768

  Final embedding: 768 dims = 384 (sum) + 384 (max)


In [7]:
# Memory Management Configuration
MEMORY_CONFIG = {
    "post_batch_size": 32,        # Encode N posts at a time
    "subject_cache_interval": 10,  # Clear GPU cache every N subjects
    "use_no_grad": True,           # Disable gradient tracking
    "move_to_cpu_immediately": True # Move results to CPU after computation
}

print("✓ Memory configuration:")
for k, v in MEMORY_CONFIG.items():
    print(f"  {k}: {v}")

✓ Memory configuration:
  post_batch_size: 32
  subject_cache_interval: 10
  use_no_grad: True
  move_to_cpu_immediately: True


In [8]:
import gc

def clear_gpu_cache():
    """Clear GPU cache and run garbage collection."""
    if DEVICE == "mps":
        torch.mps.empty_cache()
    elif DEVICE == "cuda":
        torch.cuda.empty_cache()
    gc.collect()

print("✓ GPU cache clearing utility defined")

✓ GPU cache clearing utility defined


## Section 1: Load Training Data

Extract 486 training subjects with posts and concept labels

In [9]:
# Helper functions for XML parsing
WHITESPACE_RE = re.compile(r"\s+")

def normalize_text(text):
    """Normalize text by removing null chars and extra whitespace."""
    if not text:
        return ""
    text = text.replace("\u0000", "")
    text = WHITESPACE_RE.sub(" ", text).strip()
    return text

def extract_posts_from_xml(xml_path, min_chars=10):
    """Extract posts from a single XML file."""
    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()
    except Exception as e:
        print(f"WARNING: Failed to parse {xml_path}: {e}")
        return []
    
    posts = []
    for writing in root.findall("WRITING"):
        title = writing.findtext("TITLE") or ""
        text = writing.findtext("TEXT") or ""
        
        combined = normalize_text(f"{title} {text}".strip())
        if len(combined) >= min_chars:
            posts.append(combined)
    
    return posts

print("✓ Helper functions defined")

✓ Helper functions defined


In [10]:
# Parse training XML files
print("Loading training data...")
start_time = time.time()

train_data = []

# Process positive examples
print("  Processing positive examples...")
pos_files = glob.glob(os.path.join(POS_DIR, "**", "*.xml"), recursive=True)
for xml_file in tqdm(pos_files, desc="Processing positive examples"):
    filename = os.path.basename(xml_file)
    match = re.match(r"train_(subject\d+)_\d+\.xml", filename)
    if match:
        subject_id = match.group(1)
        posts = extract_posts_from_xml(xml_file)
        for post in posts:
            train_data.append({
                "subject_id": subject_id,
                "label": 1,  # Positive (depression)
                "text": post
            })

print(f"  Loaded {sum(d['label'] == 1 for d in train_data)} posts from positive subjects")

# Process negative examples
print("  Processing negative examples...")
neg_files = glob.glob(os.path.join(NEG_DIR, "**", "*.xml"), recursive=True)
for xml_file in tqdm(neg_files, desc="Processing negative examples"):
    filename = os.path.basename(xml_file)
    match = re.match(r"train_(subject\d+)_\d+\.xml", filename)
    if match:
        subject_id = match.group(1)
        posts = extract_posts_from_xml(xml_file)
        for post in posts:
            train_data.append({
                "subject_id": subject_id,
                "label": 0,  # Negative (control)
                "text": post
            })

train_posts_df = pd.DataFrame(train_data)

print(f"\n✓ Loaded training data in {time.time()-start_time:.1f}s")
print(f"  Total posts: {len(train_posts_df):,}")
print(f"  Unique subjects: {train_posts_df['subject_id'].nunique()}")
print(f"  Label distribution:")
print(train_posts_df.groupby('label')['subject_id'].nunique())

Loading training data...
  Processing positive examples...


Processing positive examples: 100%|██████████| 830/830 [00:00<00:00, 2406.80it/s]


  Loaded 29868 posts from positive subjects
  Processing negative examples...


Processing negative examples: 100%|██████████| 4031/4031 [00:02<00:00, 1702.87it/s]



✓ Loaded training data in 2.9s
  Total posts: 286,740
  Unique subjects: 486
  Label distribution:
label
0    403
1     83
Name: subject_id, dtype: int64


In [11]:
# Load concept labels from questionnaires
print("Loading concept labels...")

concepts_df = pd.read_csv(CONCEPTS_FILE)
concepts_df["subject_id"] = concepts_df["Subject"].str.replace("train_", "", regex=True)

# Binarize concept values
concept_cols = [col for col in concepts_df.columns if col in CONCEPT_NAMES]
for col in concept_cols:
    concepts_df[col] = (concepts_df[col] > 0).astype(int)

print(f"✓ Loaded concept labels for {len(concepts_df)} subjects")

Loading concept labels...
✓ Loaded concept labels for 486 subjects


## Section 2: Load Test Data

Load all 401 test subjects from test folder (will be used entirely as test set)

In [12]:
# Extract test ZIP files to temporary directory
print("Extracting test data...")
temp_dir = tempfile.mkdtemp(prefix="test_chunks_")
print(f"  Temp directory: {temp_dir}")

for i in range(1, 11):
    zip_path = os.path.join(TEST_DIR, f"chunk {i}.zip")
    if os.path.exists(zip_path):
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(os.path.join(temp_dir, f"chunk_{i}"))
        if i % 3 == 0:
            print(f"  Extracted chunk {i}/10")

print("✓ Test data extracted")

Extracting test data...
  Temp directory: /var/folders/gb/m6c_r5xx6_14p7mlfjwk29900000gn/T/test_chunks_5djc6uf6
  Extracted chunk 3/10
  Extracted chunk 6/10
  Extracted chunk 9/10
✓ Test data extracted


In [13]:
# Load test labels
test_labels_df = pd.read_csv(TEST_LABELS, sep='\t', header=None, names=['subject_id', 'label'])
test_labels_df['subject_id'] = test_labels_df['subject_id'].str.strip()

print(f"✓ Loaded test labels for {len(test_labels_df)} subjects")
print(f"  Label distribution:")
print(test_labels_df['label'].value_counts())

✓ Loaded test labels for 401 subjects
  Label distribution:
label
0    349
1     52
Name: count, dtype: int64


In [14]:
# Parse test XML files
print("Loading test posts...")
test_data = []

test_xml_files = glob.glob(os.path.join(temp_dir, "**", "*.xml"), recursive=True)
print(f"  Found {len(test_xml_files)} XML files")

for xml_file in test_xml_files:
    filename = os.path.basename(xml_file)
    match = re.match(r"(test_subject\d+)_\d+\.xml", filename)
    if match:
        subject_id = match.group(1)
        label_row = test_labels_df[test_labels_df['subject_id'] == subject_id]
        if len(label_row) > 0:
            label = label_row.iloc[0]['label']
            posts = extract_posts_from_xml(xml_file)
            for post in posts:
                test_data.append({
                    "subject_id": subject_id,
                    "label": label,
                    "text": post
                })

test_posts_df = pd.DataFrame(test_data)

print(f"✓ Loaded test posts")
print(f"  Total posts: {len(test_posts_df):,}")
print(f"  Unique subjects: {test_posts_df['subject_id'].nunique()}")

Loading test posts...
  Found 4010 XML files
✓ Loaded test posts
  Total posts: 229,746
  Unique subjects: 401


In [15]:
# Split TRAINING data into train and validation (80/20)
print("Splitting training data into train (80%) and validation (20%)...")

train_subjects = train_posts_df.groupby('subject_id')['label'].first().reset_index()

train_subjects_final, val_subjects = train_test_split(
    train_subjects['subject_id'],
    test_size=0.2,
    stratify=train_subjects['label'],
    random_state=SEED
)

# Create new train dataframe with only 80% of subjects
train_posts_df_final = train_posts_df[train_posts_df['subject_id'].isin(train_subjects_final)].copy()

# Create validation dataframe from remaining 20% of training subjects
val_posts_df = train_posts_df[train_posts_df['subject_id'].isin(val_subjects)].copy()

# Keep ALL test data as test set (no split)
test_posts_df_final = test_posts_df.copy()

print(f"✓ Split complete")
print(f"  Training: {train_posts_df_final['subject_id'].nunique()} subjects (80% of original train)")
print(f"  Validation: {val_posts_df['subject_id'].nunique()} subjects (20% of original train)")
print(f"  Test: {test_posts_df_final['subject_id'].nunique()} subjects (100% of test folder)")

# Show label distributions
print(f"\n  Training label distribution:")
print(train_posts_df_final.groupby('label')['subject_id'].nunique())
print(f"\n  Validation label distribution:")
print(val_posts_df.groupby('label')['subject_id'].nunique())
print(f"\n  Test label distribution:")
print(test_posts_df_final.groupby('label')['subject_id'].nunique())

Splitting training data into train (80%) and validation (20%)...
✓ Split complete
  Training: 388 subjects (80% of original train)
  Validation: 98 subjects (20% of original train)
  Test: 401 subjects (100% of test folder)

  Training label distribution:
label
0    322
1     66
Name: subject_id, dtype: int64

  Validation label distribution:
label
0    81
1    17
Name: subject_id, dtype: int64

  Test label distribution:
label
0    349
1     52
Name: subject_id, dtype: int64


## Section 3: SBERT Setup & Concept Embeddings

In [16]:
# Load SBERT model
print(f"Loading SBERT model: {HYPERPARAMS['sbert_model']}")
sbert_model = SentenceTransformer(HYPERPARAMS['sbert_model'])
sbert_model = sbert_model.to(DEVICE)

print(f"✓ SBERT model loaded on {DEVICE}")
print(f"  Embedding dimension: {sbert_model.get_sentence_embedding_dimension()}")

Loading SBERT model: all-MiniLM-L6-v2
✓ SBERT model loaded on mps
  Embedding dimension: 384


In [17]:
# Create concept embeddings
print(f"Creating embeddings for {N_CONCEPTS} concepts...")
concept_embeddings = sbert_model.encode(
    CONCEPT_NAMES,
    convert_to_tensor=True,
    show_progress_bar=False
)

print(f"✓ Concept embeddings created")
print(f"  Shape: {concept_embeddings.shape}")

Creating embeddings for 21 concepts...
✓ Concept embeddings created
  Shape: torch.Size([21, 384])


In [18]:
def retrieve_top_k_posts_sum(subject_id, posts_df, concept_embs, sbert, k=50, batch_size=32, debug=False):
    """
    Retrieve top-k posts for a subject based on SUM of concept similarities.
    OPTIMIZED: Uses batching to prevent memory exhaustion.
    """
    subj_posts = posts_df[posts_df['subject_id'] == subject_id]['text'].tolist()

    if len(subj_posts) == 0:
        return []

    if len(subj_posts) <= k:
        if len(subj_posts) < k:
            extra_needed = k - len(subj_posts)
            padding = list(np.random.choice(subj_posts, size=extra_needed, replace=True))
            return subj_posts + padding
        else:
            return subj_posts

    # Batch encoding to prevent memory issues
    sum_sim_scores = []

    with torch.no_grad():  # Disable gradient tracking
        for i in range(0, len(subj_posts), batch_size):
            batch_posts = subj_posts[i:i + batch_size]

            # Encode batch
            batch_embeddings = sbert.encode(
                batch_posts,
                convert_to_tensor=True,
                show_progress_bar=False
            )

            # Compute similarities for this batch
            cos_scores = util.cos_sim(batch_embeddings, concept_embs)  # [batch, 21]
            batch_sum_scores = cos_scores.sum(dim=1).cpu().numpy()  # Move to CPU immediately

            sum_sim_scores.extend(batch_sum_scores)

            # Clear references
            del batch_embeddings, cos_scores, batch_sum_scores

    sum_sim_scores = np.array(sum_sim_scores)

    if debug:
        print("\n" + "="*60)
        print(f"[DEBUG][SUM] Subject: {subject_id}")
        print(f"[DEBUG][SUM] Total posts: {len(subj_posts)}")
        print("[DEBUG][SUM] Sum similarity stats:")
        print(f"  min={sum_sim_scores.min():.4f} "
              f"max={sum_sim_scores.max():.4f} "
              f"mean={sum_sim_scores.mean():.4f} "
              f"std={sum_sim_scores.std():.4f}")

        top_idx_sorted = np.argsort(-sum_sim_scores)
        print(f"\n[DEBUG][SUM] Top-{DEBUG_TOP_N_POSTS} retrieved posts:")
        for rank, i in enumerate(top_idx_sorted[:DEBUG_TOP_N_POSTS]):
            print(f"\n  Rank {rank+1}")
            print(f"  Score: {sum_sim_scores[i]:.4f}")
            print(f"  Text: {subj_posts[i][:200]}")

    # Select top-k posts
    top_k_indices = np.argpartition(-sum_sim_scores, range(min(k, len(subj_posts))))[:k]

    return [subj_posts[i] for i in top_k_indices]

print("✓ SUM-based post retrieval function defined")

✓ SUM-based post retrieval function defined


In [19]:
def retrieve_top_k_posts_max(subject_id, posts_df, concept_embs, sbert, k=50, batch_size=32, debug=False):
    """
    Retrieve top-k posts for a subject based on MAX of concept similarities.
    
    Key difference from sum-based:
    - For each post, takes MAX similarity across all 21 concepts
    - Selects posts that are highly relevant to at least ONE concept
    """
    subj_posts = posts_df[posts_df['subject_id'] == subject_id]['text'].tolist()

    if len(subj_posts) == 0:
        return []

    if len(subj_posts) <= k:
        if len(subj_posts) < k:
            extra_needed = k - len(subj_posts)
            padding = list(np.random.choice(subj_posts, size=extra_needed, replace=True))
            return subj_posts + padding
        else:
            return subj_posts

    # Batch encoding to prevent memory issues
    max_sim_scores = []

    with torch.no_grad():  # Disable gradient tracking
        for i in range(0, len(subj_posts), batch_size):
            batch_posts = subj_posts[i:i + batch_size]

            # Encode batch
            batch_embeddings = sbert.encode(
                batch_posts,
                convert_to_tensor=True,
                show_progress_bar=False
            )

            # Compute similarities for this batch
            cos_scores = util.cos_sim(batch_embeddings, concept_embs)  # [batch, 21]
            # KEY DIFFERENCE: Take MAX instead of SUM
            batch_max_scores = cos_scores.max(dim=1)[0].cpu().numpy()  # [0] gets values, not indices

            max_sim_scores.extend(batch_max_scores)

            # Clear references
            del batch_embeddings, cos_scores, batch_max_scores

    max_sim_scores = np.array(max_sim_scores)

    if debug:
        print("\n" + "="*60)
        print(f"[DEBUG][MAX] Subject: {subject_id}")
        print(f"[DEBUG][MAX] Total posts: {len(subj_posts)}")
        print("[DEBUG][MAX] Max similarity stats:")
        print(f"  min={max_sim_scores.min():.4f} "
              f"max={max_sim_scores.max():.4f} "
              f"mean={max_sim_scores.mean():.4f} "
              f"std={max_sim_scores.std():.4f}")

        top_idx_sorted = np.argsort(-max_sim_scores)
        print(f"\n[DEBUG][MAX] Top-{DEBUG_TOP_N_POSTS} retrieved posts:")
        for rank, i in enumerate(top_idx_sorted[:DEBUG_TOP_N_POSTS]):
            print(f"\n  Rank {rank+1}")
            print(f"  Score: {max_sim_scores[i]:.4f}")
            print(f"  Text: {subj_posts[i][:200]}")

    # Select top-k posts
    top_k_indices = np.argpartition(-max_sim_scores, range(min(k, len(subj_posts))))[:k]

    return [subj_posts[i] for i in top_k_indices]

print("✓ MAX-based post retrieval function defined")

✓ MAX-based post retrieval function defined


In [20]:
def encode_and_attention_pool_sum(selected_posts_dict, sbert, concept_embs,
                                   normalize=True, debug=False):
    """
    Encode posts and pool using SUM of concept similarities for attention.
    OPTIMIZED: Includes memory management for stability.
    """
    subject_ids = list(selected_posts_dict.keys())
    pooled_embeddings = []

    with torch.no_grad():  # Disable gradient tracking
        for idx, subject_id in enumerate(subject_ids):
            posts = selected_posts_dict[subject_id]

            # Handle empty posts
            if len(posts) == 0:
                print(f"WARNING: No posts for subject {subject_id}, using zero embedding")
                pooled_embeddings.append(np.zeros(384))
                continue

            # Filter out empty posts
            posts = [p for p in posts if p.strip()]
            if len(posts) == 0:
                print(f"WARNING: All posts empty for subject {subject_id}, using zero embedding")
                pooled_embeddings.append(np.zeros(384))
                continue

            # Encode posts
            post_embs = sbert.encode(
                posts,
                convert_to_tensor=True,
                show_progress_bar=False
            )

            if post_embs.shape[0] == 0 or post_embs.shape[1] == 0:
                print(f"WARNING: Empty embeddings for subject {subject_id}, using zero embedding")
                pooled_embeddings.append(np.zeros(384))
                continue

            # Compute similarity to concepts
            cos_scores = util.cos_sim(post_embs, concept_embs)

            # Sum over concepts
            post_scores = cos_scores.sum(dim=1)

            # Remove negative similarities
            post_scores = torch.clamp(post_scores, min=0.0)

            # Attention weights
            TEMPERATURE = 0.2  
            attn_weights = torch.softmax(post_scores / TEMPERATURE, dim=0)

            if debug and idx < DEBUG_N_SUBJECTS:
                print("\n" + "="*60)
                print(f"[DEBUG][SUM-ATTN] Subject: {subject_id}")
                attn_np = attn_weights.cpu().numpy()
                print("[DEBUG][SUM-ATTN] Weight stats:")
                print(f"  min={attn_np.min():.6f} "
                      f"max={attn_np.max():.6f} "
                      f"mean={attn_np.mean():.6f}")

            # Weighted sum pooling
            pooled = torch.sum(attn_weights.unsqueeze(1) * post_embs, dim=0)
            pooled_embeddings.append(pooled.cpu().numpy())

            # Clean up GPU memory
            del post_embs, cos_scores, attn_weights, pooled

    return np.vstack(pooled_embeddings), subject_ids

print("✓ SUM-based attention pooling function defined")

✓ SUM-based attention pooling function defined


In [21]:
def encode_and_attention_pool_max(selected_posts_dict, sbert, concept_embs,
                                   normalize=True, debug=False):
    """
    Encode posts and pool using MAX of concept similarities for attention.
    
    For each post:
    1. Compute similarities to all 21 concepts
    2. Take MAX similarity as the post's relevance score
    3. Use softmax(max_scores / temperature) for attention weights
    4. Weighted sum pooling to create final embedding
    """
    subject_ids = list(selected_posts_dict.keys())
    pooled_embeddings = []

    with torch.no_grad():  # Disable gradient tracking
        for idx, subject_id in enumerate(subject_ids):
            posts = selected_posts_dict[subject_id]

            # Handle empty posts
            if len(posts) == 0:
                print(f"WARNING: No posts for subject {subject_id}, using zero embedding")
                pooled_embeddings.append(np.zeros(384))
                continue

            # Filter out empty posts
            posts = [p for p in posts if p.strip()]
            if len(posts) == 0:
                print(f"WARNING: All posts empty for subject {subject_id}, using zero embedding")
                pooled_embeddings.append(np.zeros(384))
                continue

            # Encode posts
            post_embs = sbert.encode(
                posts,
                convert_to_tensor=True,
                show_progress_bar=False
            )

            if post_embs.shape[0] == 0 or post_embs.shape[1] == 0:
                print(f"WARNING: Empty embeddings for subject {subject_id}, using zero embedding")
                pooled_embeddings.append(np.zeros(384))
                continue

            # Compute similarity to concepts
            cos_scores = util.cos_sim(post_embs, concept_embs)

            # KEY DIFFERENCE: Take MAX instead of SUM
            post_scores = cos_scores.max(dim=1)[0]  # [0] gets values, not indices

            # Remove negative similarities
            post_scores = torch.clamp(post_scores, min=0.0)

            # Attention weights
            TEMPERATURE = 0.2  
            attn_weights = torch.softmax(post_scores / TEMPERATURE, dim=0)

            if debug and idx < DEBUG_N_SUBJECTS:
                print("\n" + "="*60)
                print(f"[DEBUG][MAX-ATTN] Subject: {subject_id}")
                attn_np = attn_weights.cpu().numpy()
                print("[DEBUG][MAX-ATTN] Weight stats:")
                print(f"  min={attn_np.min():.6f} "
                      f"max={attn_np.max():.6f} "
                      f"mean={attn_np.mean():.6f}")

            # Weighted sum pooling
            pooled = torch.sum(attn_weights.unsqueeze(1) * post_embs, dim=0)
            pooled_embeddings.append(pooled.cpu().numpy())

            # Clean up GPU memory
            del post_embs, cos_scores, attn_weights, pooled

    return np.vstack(pooled_embeddings), subject_ids

print("✓ MAX-based attention pooling function defined")

✓ MAX-based attention pooling function defined


## Section 4: Dual Post Retrieval (Sum + Max)

Retrieve top-50 posts using BOTH methods for each subject

In [22]:
# Retrieve top-k posts for all subjects using BOTH methods
print(f"Retrieving top-{HYPERPARAMS['k_posts']} posts using SUM and MAX methods...")
print("⏰ This will take ~2x longer due to dual retrieval")
start_time = time.time()

# Training subjects (80% of original training data)
print("\n  Processing training subjects (80% of train data)...")
train_sum_selected = {}
train_max_selected = {}
train_subjects = train_posts_df_final['subject_id'].unique()

for idx, subject_id in enumerate(tqdm(train_subjects, desc="Train subjects (sum+max)")):
    # Method 1: Sum-based retrieval
    sum_posts = retrieve_top_k_posts_sum(
        subject_id,
        train_posts_df_final,
        concept_embeddings,
        sbert_model,
        k=HYPERPARAMS['k_posts'],
        batch_size=MEMORY_CONFIG['post_batch_size'],
        debug=(DEBUG and idx < DEBUG_N_SUBJECTS)
    )
    train_sum_selected[subject_id] = sum_posts
    
    # Method 2: Max-based retrieval
    max_posts = retrieve_top_k_posts_max(
        subject_id,
        train_posts_df_final,
        concept_embeddings,
        sbert_model,
        k=HYPERPARAMS['k_posts'],
        batch_size=MEMORY_CONFIG['post_batch_size'],
        debug=(DEBUG and idx < DEBUG_N_SUBJECTS)
    )
    train_max_selected[subject_id] = max_posts

    # Clear GPU cache periodically
    if (idx + 1) % MEMORY_CONFIG['subject_cache_interval'] == 0:
        clear_gpu_cache()

# Validation subjects (20% of original training data)
print("\n  Processing validation subjects (20% of train data)...")
val_sum_selected = {}
val_max_selected = {}
val_subjects = val_posts_df['subject_id'].unique()

for idx, subject_id in enumerate(tqdm(val_subjects, desc="Val subjects (sum+max)")):
    # Method 1: Sum-based retrieval
    sum_posts = retrieve_top_k_posts_sum(
        subject_id,
        val_posts_df,
        concept_embeddings,
        sbert_model,
        k=HYPERPARAMS['k_posts'],
        batch_size=MEMORY_CONFIG['post_batch_size'],
        debug=(DEBUG and idx < DEBUG_N_SUBJECTS)
    )
    val_sum_selected[subject_id] = sum_posts
    
    # Method 2: Max-based retrieval
    max_posts = retrieve_top_k_posts_max(
        subject_id,
        val_posts_df,
        concept_embeddings,
        sbert_model,
        k=HYPERPARAMS['k_posts'],
        batch_size=MEMORY_CONFIG['post_batch_size'],
        debug=(DEBUG and idx < DEBUG_N_SUBJECTS)
    )
    val_max_selected[subject_id] = max_posts

    if (idx + 1) % MEMORY_CONFIG['subject_cache_interval'] == 0:
        clear_gpu_cache()

# Test subjects (100% of test folder)
print("\n  Processing test subjects (100% of test folder)...")
test_sum_selected = {}
test_max_selected = {}
test_subjects = test_posts_df_final['subject_id'].unique()

for idx, subject_id in enumerate(tqdm(test_subjects, desc="Test subjects (sum+max)")):
    # Method 1: Sum-based retrieval
    sum_posts = retrieve_top_k_posts_sum(
        subject_id,
        test_posts_df_final,
        concept_embeddings,
        sbert_model,
        k=HYPERPARAMS['k_posts'],
        batch_size=MEMORY_CONFIG['post_batch_size'],
        debug=(DEBUG and idx < DEBUG_N_SUBJECTS)
    )
    test_sum_selected[subject_id] = sum_posts
    
    # Method 2: Max-based retrieval
    max_posts = retrieve_top_k_posts_max(
        subject_id,
        test_posts_df_final,
        concept_embeddings,
        sbert_model,
        k=HYPERPARAMS['k_posts'],
        batch_size=MEMORY_CONFIG['post_batch_size'],
        debug=(DEBUG and idx < DEBUG_N_SUBJECTS)
    )
    test_max_selected[subject_id] = max_posts

    if (idx + 1) % MEMORY_CONFIG['subject_cache_interval'] == 0:
        clear_gpu_cache()

# Final cache clear
clear_gpu_cache()

print(f"\n✓ Dual post retrieval complete in {time.time()-start_time:.1f}s ({(time.time()-start_time)/60:.1f} min)")
print(f"  Processed {len(train_subjects) + len(val_subjects) + len(test_subjects)} subjects with BOTH methods")

Retrieving top-50 posts using SUM and MAX methods...
⏰ This will take ~2x longer due to dual retrieval

  Processing training subjects (80% of train data)...


Train subjects (sum+max):   0%|          | 0/388 [00:00<?, ?it/s]


[DEBUG][SUM] Subject: subject6760
[DEBUG][SUM] Total posts: 124
[DEBUG][SUM] Sum similarity stats:
  min=-0.9500 max=5.2591 mean=1.5585 std=1.3026

[DEBUG][SUM] Top-5 retrieved posts:

  Rank 1
  Score: 5.2591
  Text: Not feeling negative could be considered as better.

  Rank 2
  Score: 4.8450
  Text: The 12 Worst Habits for Your Mental Health

  Rank 3
  Score: 4.2735
  Text: Can you explain?

  Rank 4
  Score: 4.2102
  Text: A roller-coaster in which I always seem to be lost in the past or in the future, never living the moment.

  Rank 5
  Score: 3.8703
  Text: Selfish motives seem to make you succeed. I'm shocked!


Train subjects (sum+max):   0%|          | 1/388 [00:01<11:50,  1.84s/it]


[DEBUG][MAX] Subject: subject6760
[DEBUG][MAX] Total posts: 124
[DEBUG][MAX] Max similarity stats:
  min=0.0221 max=0.6737 mean=0.2090 std=0.0993

[DEBUG][MAX] Top-5 retrieved posts:

  Rank 1
  Score: 0.6737
  Text: My therapist asked me to try and be an imaginary observer about how critical I'm about myself. She thinks that even if you can't stop being hyper-self-critical, being aware of it helps. (PS: Is your n

  Rank 2
  Score: 0.4208
  Text: The 12 Worst Habits for Your Mental Health

  Rank 3
  Score: 0.4175
  Text: It's sometimes hard for me to do imaginary tasks - e.g., the containment task in mindfulness is not easy for me. :)

  Rank 4
  Score: 0.4051
  Text: Must confess. I do both.

  Rank 5
  Score: 0.4041
  Text: Not feeling negative could be considered as better.

[DEBUG][SUM] Subject: subject7326
[DEBUG][SUM] Total posts: 118
[DEBUG][SUM] Sum similarity stats:
  min=-1.0593 max=3.1569 mean=1.0799 std=0.8906

[DEBUG][SUM] Top-5 retrieved posts:

  Rank 1
  Score: 3.156

Train subjects (sum+max):   1%|          | 3/388 [00:02<05:24,  1.18it/s]


[DEBUG][MAX] Subject: subject7326
[DEBUG][MAX] Total posts: 118
[DEBUG][MAX] Max similarity stats:
  min=0.0131 max=0.4619 mean=0.1764 std=0.0727

[DEBUG][MAX] Top-5 retrieved posts:

  Rank 1
  Score: 0.4619
  Text: Thank you for sharing. I'll look into that link deeper tomorrow, but I always thought that maybe it's my biological clock that is not working correctly rather than the actual sleep. I can sleep during

  Rank 2
  Score: 0.3985
  Text: I have similar experiences. I have often HUGE problems falling asleep and lying around waiting until my body decides it's ok now to fall asleep is just super annoying and exhausting for me. I'll mostl

  Rank 3
  Score: 0.3252
  Text: "actively sabotage" lol. He offered her food. That's about it. He didn't fill her food with hidden chocolate and neither did he shove down pizza down her throat when she was asleep. There's NO point o

  Rank 4
  Score: 0.3210
  Text: History. Actually I'm in 2nd semester already, but better late than never is 

Train subjects (sum+max):  10%|█         | 40/388 [01:22<11:59,  2.07s/it]


KeyboardInterrupt: 

## Section 5: Dual Embedding Generation & Concatenation

Create sum-based and max-based embeddings, then concatenate to 768 dims

In [ ]:
# Encode and pool for all splits using BOTH methods
print("Encoding and pooling embeddings (SUM + MAX methods, memory-optimized)...")
print("Final embeddings will be 768-dim (384 sum + 384 max)\n")

start_time = time.time()

# ==================== TRAINING SET ====================
print("  Training set...")
# Sum-based embeddings (384 dims)
X_train_sum, train_subject_ids = encode_and_attention_pool_sum(
    train_sum_selected,
    sbert_model,
    concept_embeddings,
    normalize=True,
    debug=DEBUG
)
clear_gpu_cache()
print(f"    X_train_sum shape: {X_train_sum.shape}")

# Max-based embeddings (384 dims)
X_train_max, _ = encode_and_attention_pool_max(
    train_max_selected,
    sbert_model,
    concept_embeddings,
    normalize=True,
    debug=DEBUG
)
clear_gpu_cache()
print(f"    X_train_max shape: {X_train_max.shape}")

# Concatenate: 768 dims total
X_train = np.hstack([X_train_sum, X_train_max])
print(f"    X_train (concatenated) shape: {X_train.shape}\n")

# ==================== VALIDATION SET ====================
print("  Validation set...")
# Sum-based embeddings (384 dims)
X_val_sum, val_subject_ids = encode_and_attention_pool_sum(
    val_sum_selected,
    sbert_model,
    concept_embeddings,
    normalize=True
)
clear_gpu_cache()
print(f"    X_val_sum shape: {X_val_sum.shape}")

# Max-based embeddings (384 dims)
X_val_max, _ = encode_and_attention_pool_max(
    val_max_selected,
    sbert_model,
    concept_embeddings,
    normalize=True
)
clear_gpu_cache()
print(f"    X_val_max shape: {X_val_max.shape}")

# Concatenate: 768 dims total
X_val = np.hstack([X_val_sum, X_val_max])
print(f"    X_val (concatenated) shape: {X_val.shape}\n")

# ==================== TEST SET ====================
print("  Test set...")
# Sum-based embeddings (384 dims)
X_test_sum, test_subject_ids = encode_and_attention_pool_sum(
    test_sum_selected,
    sbert_model,
    concept_embeddings,
    normalize=True
)
clear_gpu_cache()
print(f"    X_test_sum shape: {X_test_sum.shape}")

# Max-based embeddings (384 dims)
X_test_max, _ = encode_and_attention_pool_max(
    test_max_selected,
    sbert_model,
    concept_embeddings,
    normalize=True
)
clear_gpu_cache()
print(f"    X_test_max shape: {X_test_max.shape}")

# Concatenate: 768 dims total
X_test = np.hstack([X_test_sum, X_test_max])
print(f"    X_test (concatenated) shape: {X_test.shape}\n")

print(f"✓ Encoding complete in {time.time()-start_time:.1f}s ({(time.time()-start_time)/60:.1f} min)")
print(f"\n  Final embedding dimensions: {HYPERPARAMS['final_embedding_dim']} = {HYPERPARAMS['embedding_dim']} (sum) + {HYPERPARAMS['embedding_dim']} (max)")

## Section 6: Build Concept Matrices and Labels

In [ ]:
# Build concept matrices and label vectors
print("Building concept matrices and labels...")

# Training: get concepts from questionnaires (80% of training data)
C_train = []
y_train = []
for subject_id in train_subject_ids:
    label = train_posts_df_final[train_posts_df_final['subject_id'] == subject_id]['label'].iloc[0]
    y_train.append(label)
    
    concept_row = concepts_df[concepts_df['subject_id'] == subject_id]
    if len(concept_row) > 0:
        concepts = concept_row[concept_cols].values[0]
    else:
        concepts = np.zeros(N_CONCEPTS)
    C_train.append(concepts)

C_train = np.array(C_train, dtype=np.float32)
y_train = np.array(y_train, dtype=np.float32)

# Validation: get concepts from questionnaires (20% of training data)
C_val = []
y_val = []
for subject_id in val_subject_ids:
    label = val_posts_df[val_posts_df['subject_id'] == subject_id]['label'].iloc[0]
    y_val.append(label)
    
    concept_row = concepts_df[concepts_df['subject_id'] == subject_id]
    if len(concept_row) > 0:
        concepts = concept_row[concept_cols].values[0]
    else:
        concepts = np.zeros(N_CONCEPTS)
    C_val.append(concepts)

C_val = np.array(C_val, dtype=np.float32)
y_val = np.array(y_val, dtype=np.float32)

# Test: zeros for concepts (no ground truth available)
C_test = np.zeros((len(test_subject_ids), N_CONCEPTS), dtype=np.float32)
y_test = []
for subject_id in test_subject_ids:
    label = test_posts_df_final[test_posts_df_final['subject_id'] == subject_id]['label'].iloc[0]
    y_test.append(label)
y_test = np.array(y_test, dtype=np.float32)

print("✓ Matrices built")
print(f"  Train: X={X_train.shape}, C={C_train.shape}, y={y_train.shape}")
print(f"  Val:   X={X_val.shape}, C={C_val.shape}, y={y_val.shape}")
print(f"  Test:  X={X_test.shape}, C={C_test.shape}, y={y_test.shape}")
print(f"\n  Training label distribution: {np.bincount(y_train.astype(int))}")
print(f"  Validation label distribution: {np.bincount(y_val.astype(int))}")
print(f"  Test label distribution: {np.bincount(y_test.astype(int))}")

## Section 7: Compute Class Weights

In [ ]:
# Compute class weights for imbalanced dataset
n_negative = int(np.sum(y_train == 0))
n_positive = int(np.sum(y_train == 1))
pos_weight = n_negative / n_positive

print(f"Class imbalance:")
print(f"  Negative samples: {n_negative}")
print(f"  Positive samples: {n_positive}")
print(f"  Ratio: 1:{pos_weight:.2f}")
print(f"  Computed pos_weight: {pos_weight:.4f}")

## Section 8: Save All Datasets

Save everything for fast loading by training pipelines

In [ ]:
# Save processed datasets to disk
print("Saving datasets...")

# Save numpy arrays
np.savez_compressed(
    os.path.join(SAVE_DIR, "train_data.npz"),
    X=X_train,
    C=C_train,
    y=y_train,
    subject_ids=np.array(train_subject_ids)
)

np.savez_compressed(
    os.path.join(SAVE_DIR, "val_data.npz"),
    X=X_val,
    C=C_val,
    y=y_val,
    subject_ids=np.array(val_subject_ids)
)

np.savez_compressed(
    os.path.join(SAVE_DIR, "test_data.npz"),
    X=X_test,
    C=C_test,
    y=y_test,
    subject_ids=np.array(test_subject_ids)
)

# Save class weights info
class_info = {
    "n_positive": n_positive,
    "n_negative": n_negative,
    "pos_weight": float(pos_weight)
}

with open(os.path.join(SAVE_DIR, "class_weights.json"), 'w') as f:
    json.dump(class_info, f, indent=4)

print(f"✓ Datasets saved to {SAVE_DIR}")
print(f"  train_data.npz: {X_train.shape[0]} samples, {X_train.shape[1]} dims (768)")
print(f"  val_data.npz:   {X_val.shape[0]} samples, {X_val.shape[1]} dims (768)")
print(f"  test_data.npz:  {X_test.shape[0]} samples, {X_test.shape[1]} dims (768)")
print(f"  class_weights.json")

## Section 9: Cleanup

In [ ]:
# Clean up temporary directory
try:
    shutil.rmtree(temp_dir)
    print(f"✓ Cleaned up temporary directory: {temp_dir}")
except Exception as e:
    print(f"⚠ Failed to clean up temporary directory: {e}")

In [ ]:
print("\n" + "="*70)
print("      LONG ALTERNATIVE DATASET PREPARATION COMPLETE")
print("="*70)
print("\nSaved files:")
print(f"  {SAVE_DIR}/train_data.npz")
print(f"  {SAVE_DIR}/val_data.npz")
print(f"  {SAVE_DIR}/test_data.npz")
print(f"  {SAVE_DIR}/class_weights.json")
print("\nData split strategy:")
print("  - Training: 80% of train folder (~389 subjects)")
print("  - Validation: 20% of train folder (~97 subjects)")
print("  - Test: 100% of test folder (401 subjects)")
print("\nKey differences from original:")
print("  - Uses BOTH sum and max of concept similarities")
print("  - Retrieves posts separately for each method")
print("  - Concatenates embeddings: 384 (sum) + 384 (max) = 768 dims")
print("  - Captures both multi-concept posts (sum) and single-concept experts (max)")
print("\nUse this data with CEM/CBM training notebooks!")
print("Note: Update model input_dim to 768 instead of 384")
print("="*70)